# Example 5-2: Calculating Sunrise-Sunset Times
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, p. 290-291

This notebook demonstrates calculating the sunrise and sunset times given a date and site location.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install valladopy

Import `numpy` and the relevant `valladopy` modules:

In [2]:
import numpy as np
import valladopy.constants as const
from valladopy.astro.celestial import sun
from valladopy.mathtime.julian_date import jday
from valladopy.mathtime.utils import sec2hms

## Problem Definition
---

GIVEN:&ensp;Site at $40^{\circ}\text{N}$, $0^{\circ}\text{E}$ on March 23, 1996<br>
FIND: &emsp;Sunrise and sunset times

In [3]:
year = 1996
month = 3
day = 23
latgd = np.radians(40)
lon = 0

## Solution
---

We begin by finding the Julian Date using **Algorithm 14**:

$$
JD = 367(yr)
- \text{INT}\left\{ \frac{7\left\{ yr + \text{INT} \left( \frac{mo + 9}{12} \right) \right\}}{4} \right\}
+ \text{INT}\left( \frac{275\,mo}{9} \right)
+ d + 1{,}721{,}013.5
+ \frac{h\,3600 + min\,60 + s}{86400}
$$

Which is done with the `jday` routine:

In [4]:
jd, jdfrac = jday(year, month, day)

print(f'Julian Date:\t{jd + jdfrac}')

Julian Date:	2450165.5


**Algorithm 30** shows the process for calculating sunrise and sunset times.

Begin by finding the Julian dates of the sunrise and sunset occurrences, using the JD at $0^{\text{h}}$:

$$
\begin{aligned}
JD_{sunrise} &= JD_{0^{\text{h}}} + 6/24 - \lambda/360 \\
\\
JD_{sunset} &= JD_{0^{\text{h}}} + 18/24 - \lambda/360
\end{aligned}
$$

Find the Julian centuries of UT1:

$$
T_{UT1} = \frac{JD_{UT1} - 2,451,545.0}{36,525}
$$

Find the mean anomaly and the mean and corrected ecliptic longitudes, letting $T_{TDB} \cong T_{UT1}$:

$$
\begin{aligned}
M_{\odot} &= 357.5291092^{\circ} + 35,999.05034 \ T_{UT1}
\\
\bar{\lambda}_{ecliptic \ \odot} &= 280.4606184^{\circ} + 36,000.77005361 \ T_{UT1}
\\
\lambda_{ecliptic \ \odot} &= \bar{\lambda}_{ecliptic \ \odot} + 1.914666471^{\circ} \sin(M_{\odot}) + 0.019994643 \sin(2 M_{\odot})
\end{aligned}
$$

The obliquity of the ecliptic can be approximated as:

$$
\epsilon = 23.439291^{\circ} - 0.013004 \ T_{TDB} \ \cong \ 23.439291^{\circ} - 0.013004 \ T_{UT1}
$$

And obtain the right ascension and declination with:

$$
\begin{aligned}
\tan(\alpha_{\odot}) &= \cos(\epsilon) \tan(\lambda_{ecliptic \ \odot}) \\
\\
\sin(\delta_{\odot}) &= \sin(\epsilon) \sin(\lambda_{ecliptic \ \odot})
\end{aligned}
$$

Now find the local hour angle (LHA) of the site. Using **Algorithm 28** for *sunset*:

$$
\cos(LHA_{\odot}) = \cos(LHA_{sunset}) = \frac{\cos{\varsigma} - \sin(\delta_{\odot}) \sin(\phi_{gc})}{\cos(\delta_{\odot})\cos(\phi_{gc})}
$$

and:

$$
LHA_{sunrise} = 360^{\circ} - LHA_{sunset}
$$

where $\varsigma$ is the angle between the Sun and the site and defines the position and timing of the event. For sunsrise and sunset, we use $\varsigma = 90^{\circ} 50'$ (see p. 287 for more details).

Next, get the Greenwich mean sidereal time (GMST) at $6^{\text{h}}$ using **Equation 3-46**, keeping in mind that $T_{UT1}$ contains the JD offset from earlier (making it at $6^{\text{h}}$ instead of $0^{\text{h}}$):

$$
GMST = 100.4606184^{\circ} + 36,000.77005361 \ T_{UT1} + 0.00038793 \ T^2_{UT1} - 2.6 \times 10^{-8} \ T^3_{UT1} 
$$

Finally, determine the universal time with:

$$
UT = LHA_{\odot} + \alpha_{\odot} - GMST
$$

All of this is encompassed in the `sun.rise_set` routine, which returns the sunrise and sunset universal times in **hours**:

In [5]:
sunrise, sunset = sun.rise_set(jd + jdfrac, latgd, lon)

# Convert to HMS format:
sunrise_hms = sec2hms(sunrise * const.HR2SEC)
sunset_hms = sec2hms(sunset * const.HR2SEC)

print(f'Sunrise:\t{sunrise_hms[0]} hr,\t{sunrise_hms[1]} min,\t{sunrise_hms[2]:.4f} sec')
print(f'Sunset:\t\t{sunset_hms[0]} hr,\t{sunset_hms[1]} min,\t{sunset_hms[2]:.4f} sec')

Sunrise:	5 hr,	58 min,	21.9660 sec
Sunset:		18 hr,	15 min,	17.7614 sec


These values are nearly identical to those found in the Almanac (1996).